In [1]:
# --- 1. Imports ---
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
# --- 2. Data Preparation ---
# Example: Suppose your data is in a CSV with columns:
# ['gk_x', 'gk_y', 'ball_x', 'ball_y', 'p1_x', 'p1_y', ..., 'pn_x', 'pn_y']
# Each row is a timestep; you need to create sequences.

# Load your data
data = pd.read_csv('all_coordinates-45min.csv') 
data.head()

,frame_index,ball_coords,gk_t1_coords,gk_t2_coords,team_0_players,team_1_players
0,0,NaN,NaN,NaN,"[(803.5, 634.5), (1850.5, 723.0), (1411.5, 236...","[(652.0, 437.5), (473.5, 208.5), (340.0, 656.0..."
1,1,"(891.5, 209.0)",NaN,NaN,"[(1128.0, 322.0), (1214.0, 817.5), (1534.0, 30...","[(1003.5, 282.5), (550.0, 458.0), (1080.0, 354..."
2,2,"(129.5, 595.5)",NaN,NaN,"[(1110.0, 402.0), (1551.0, 233.5), (800.0, 283...","[(1312.5, 294.0), (598.0, 259.5), (1442.0, 367..."
3,3,NaN,NaN,NaN,"[(1132.0, 614.5), (1787.5, 407.5), (1489.5, 46...","[(573.5, 384.5), (263.5, 601.0), (1240.0, 430...."
4,4,"(1778.5, 756.0)","(115.5, 319.0)","(1314.5, 246.0)","[(138.5, 847.0), (1362.0, 557.0), (250.0, 662....","[(1159.0, 535.0), (782.5, 601.0), (1587.5, 413..."


In [6]:
# Parameters
sequence_length = 10  # Number of timesteps in each sequence
num_players = (data.shape[1] - 4) // 2  # Exclude GK and ball coords

# Build sequences
def create_sequences(df, seq_len):
    X, y = [], []
    for i in range(len(df) - seq_len):
        # Input: GK, ball, team 1 players (except GK)
        seq_input = df.iloc[i:i+seq_len, 2:].values  # Exclude GK coords
        # Output: GK position at next timestep
        seq_output = df.iloc[i+seq_len, 0:2].values  # GK x, y
        X.append(seq_input)
        y.append(seq_output)
    return np.array(X), np.array(y)

X, y = create_sequences(data, sequence_length)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# --- 3. Model Definition ---
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False),
    Dense(32, activation='relu'),
    Dense(2)  # Predict GK x, y
])

model.compile(optimizer='adam', loss='mse')

In [9]:
# --- 4. Training ---
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)


ValueError: Invalid dtype: object

In [ ]:
# --- 5. Evaluation ---
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

In [ ]:
# --- 6. Prediction Example ---
y_pred = model.predict(X_test)

# Plot true vs predicted GK positions (for first 100 samples)
plt.figure(figsize=(10,5))
plt.plot(y_test[:100,0], y_test[:100,1], 'bo-', label='True GK')
plt.plot(y_pred[:100,0], y_pred[:100,1], 'rx-', label='Predicted GK')
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('GK Position Prediction')
plt.show()